In [10]:
using QuantEcon, Distributions, QuadGK, NamedTuples, BenchmarkTools

## Some More on Numerical Integration
Find $\int_0^1 sin(x) dx$ numerically

In [23]:
#Adaptive Quadrature
quadgk(x -> sin(x), 0, 1) #Returns tuple with the solution and an error bound on the integral

(0.4596976941318603, 5.551115123125783e-17)

In [3]:
dist = Normal(0.1,2)
@show dist
@show(minimum(dist),maximum(dist)) #The support
@show pdf(dist, 1.0)
#Numerical integration example
mean(dist) ≈ quadgk(x -> x * pdf(dist,x) , minimum(dist),maximum(dist))[1]

g(x) = x
quadgk(x -> g(x) * pdf(dist,x) , minimum(dist),maximum(dist))[1]

dist = Distributions.Normal{Float64}(μ=0.1, σ=2.0)
minimum(dist) = -Inf
maximum(dist) = Inf
pdf(dist, 1.0) = 0.18026348123082397


0.09999999999999992

## Other Quadrature Rules
While adaptive quadrature is typically the most efficient way to get to "machine precision", it is less appropriate when we want less precicision.  Another approach is gaussian quadrature which returns weights, $w$  and nodes, $\vec{x}$ to calculate integral $\int_a^b f(x) dx \approx f(\vec{x}) \cdot w$.

For example, to calculate the integral
$$
\int_{-\infty}^{\infty} f(x) \phi(x) dx
$$
where $\phi(x)$ is the pdf of the Normal$(\mu,\sigma^2)$,

In [4]:
n = 10
dist = Normal(0.1, 2)
nodes, weights = qnwnorm(n,[mean(dist)],[var(dist)])

#Then to get integrals of a function,
g(x) = x

#Check the calculations of the means and 
@show mean(dist) ≈ dot(g.(nodes), weights)
@show var(dist) ≈ dot(nodes.^2, weights) - dot(nodes, weights)^2
typeof(dist)


mean(dist) ≈ dot(g.(nodes), weights) = true
var(dist) ≈ dot(nodes .^ 2, weights) - dot(nodes, weights) ^ 2 = true


Distributions.Normal{Float64}

## Calculating Expectations
As an example of generic programming, we can add in expectation calculations specific to normal distributions, discrete distributions, etc.

The `expectation` function takes in a distribution, and returns an operator to calculate expectations with it

In [2]:
#This sort of general utility could be added to the Distributions library, for example.

#First, we define a specific one for normal distributions
function expectation(dist::Distributions.Normal{T}; n=20, kws...) where {T}
    nodes, weights = qnwnorm(n, [mean(dist)],[var(dist)])
    return f -> dot(f.(nodes), weights)  #Returns a function (of a function f)
end

#Then, for arbitrary distributions with a pdf function, we can  Gauss-Kronod adaptive quadrature.
#This is very precise, but does more calculations by default
function expectation(dist::D; kws...) where {D <: ContinuousUnivariateDistribution}
    #Uses Gauss-Kronod adaptive quadrature
    return f ->
        quadgk( x-> f(x) * pdf(dist,x) , minimum(dist), maximum(dist); kws...)[1] 
end

#For a discretely valued expectation, could just do the sums as a dot-product
function expectation(dist::D; kws...) where {D <: DiscreteUnivariateDistribution}
    @assert hasfinitesupport(dist) #Otherwise might need to modify to use a truncated pdf.    
    distsupport = support(dist)
    return f -> dot(pdf.(dist, distsupport), f.(distsupport))  
 end

#For convenience, we can add in a version of expectation that takes a function as the first parameter
#Less useful if you want to cach the nodes/weights, etc.
function expectation(f, dist::D; kws...) where {D <: UnivariateDistribution}
    E = expectation(dist; kws...) #Gets the appropriate expectation operator for dist
    return E(f) #calls the expectation with f
end

expectation (generic function with 4 methods)

## Using the Expectation Operator

In [22]:
#Can get an E and use it
dist = Normal(0.1, 2)
E = expectation(dist)

meanx = E(x -> x)
@show meanx

#Can define and pass in functions as well, not just anonymous
g(x) = x

varx = E(x -> x^2) - E(g)^2 #i.e. calculate the variance
@show varx

#Or can call `expectation` directly with a function as the first parameter
meanx2 = expectation(x -> x, dist)
@show meanx2

meanx = 0.09999999999999978
varx = 3.999999999999975
meanx2 = 0.09999999999999978


0.09999999999999978

In [23]:
#Try another continous distribution
#Works for any distribution supporting: pdf(dist,x) , minimum(dist), maximum(dist)
dist = Exponential(0.1)
@show dist

E = expectation(dist)
E(x -> x^2)

dist = Distributions.Exponential{Float64}(θ=0.1)


0.02

In [29]:
# Test the discrete distribution
dist = Binomial(10, 0.5)
@show dist
@show support(dist)
@show pdf.(dist, support(dist))

#The interfact for expectations is the same
E = expectation(dist)
vardist = E(n -> n^2) - E(n -> n)^2 #i.e. calculate the variance

@show vardist ≈ var(dist) #Test vs. the builtin variance for the distribution

dist = Distributions.Binomial{Float64}(n=10, p=0.5)
support(dist) = 0:10
pdf.(dist, support(dist)) = [0.000976563, 0.00976563, 0.0439453, 0.117188, 0.205078, 0.246094, 0.205078, 0.117188, 0.0439453, 0.00976563, 0.000976563]
vardist ≈ var(dist) = true


true

## Interpolation Examples

In [21]:
# Linear Interpolation
x = 0:0.1:1
@show x
y_grid = x.^2
@show y_grid
quadtest = LinInterp(x, y_grid)
finergrid = 0:0.001:1

using Plots
gr()
plot(finergrid, quadtest.(finergrid))

x = 0.0:0.1:1.0
y_grid = [0.0, 0.01, 0.04, 0.09, 0.16, 0.25, 0.36, 0.49, 0.64, 0.81, 1.0]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip5802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.4745,375.869 33.0223,375.832 33.5702,375.796 34.118,375.759 34.6659,375.723 35.2137,375.687 35.7615,375.65 36.3094,375.614 36.8572,375.577 37.4051,375.541 
 37.9529,375.505 38.5007,375.468 39.0486,375.432 39.5964,375.395 40.1443,375.359 40.6921,375.323 41.2399,375.286 41.7878,375.25 42.3356,375.213 42.8835,375.177 
 43.4313,375.141 43.9791,375.104 44.527,375.068 45.0748,375.031 45.6227,374.995 46.1705,374.959 46.7183,374.922 47.2662,374.886 47.814,374.849 48.3619,374.813 
 48.9097,374.777 49.4575,374.74 50.0054,374.704 50.5532,374.667 51.1011,374.631 51.6489,374.594 52.1968,374.558 52.7446,374.522 53.2924,374.485 53.8403,374.449 
 54.3881,374.412 54.936,374.376 55.4838,374.34 56.0316,374.303 56.5795,374.267 57.1273,374.23 57.6752,374.194 58.223,374.158 58.7708,374.121 59.3187,374.085 
 59.8665,374.048 60.4144,374.012 60.9622,373.976 61.51,373.939 62.0579,373.903 62.6057,373.866 63.1536,373.83 63.7014,373.794 64.2492,373.757 64.7971,373.721 
 65.3449,373.684 65.8928,373.648 66.4406,373.612 66.9884,373.575 67.5363,373.539 68.0841,373.502 68.632,373.466 69.1798,373.429 69.7276,373.393 70.2755,373.357 
 70.8233,373.32 71.3712,373.284 71.919,373.247 72.4668,373.211 73.0147,373.175 73.5625,373.138 74.1104,373.102 74.6582,373.065 75.2061,373.029 75.7539,372.993 
 76.3017,372.956 76.8496,372.92 77.3974,372.883 77.9453,372.847 78.4931,372.811 79.0409,372.774 79.5888,372.738 80.1366,372.701 80.6845,372.665 81.2323,372.629 
 81.7801,372.592 82.328,372.556 82.8758,372.519 83.4237,372.483 83.9715,372.447 84.5193,372.41 85.0672,372.374 85.615,372.337 86.1629,372.301 86.7107,372.265 
 87.2585,372.228 87.8064,372.119 88.3542,372.01 88.9021,371.9 89.4499,371.791 89.9977,371.682 90.5456,371.573 91.0934,371.464 91.6413,371.354 92.1891,371.245 
 92.7369,371.136 93.2848,371.027 93.8326,370.917 94.3805,370.808 94.9283,370.699 95.4761,370.59 96.024,370.481 96.5718,370.371 97.1197,370.262 97.6675,370.153 
 98.2153,370.044 98.7632,369.935 99.311,369.825 99.8589,369.716 100.407,369.607 100.955,369.498 101.502,369.388 102.05,369.279 102.598,369.17 103.146,369.061 
 103.694,368.952 104.242,368.842 104.789,368.733 105.337,368.624 105.885,368.515 106.433,368.405 106.981,368.296 107.529,368.187 108.076,368.078 108.624,367.969 
 109.172,367.859 109.72,367.75 110.268,367.641 110.816,367.532 111.364,367.423 111.911,367.313 112.459,367.204 113.007,367.095 113.555,366.986 114.103,366.876 
 114.651,366.767 115.198,366.658 115.746,366.549 116.294,366.44 116.842,366.33 117.39,366.221 117.938,366.112 118.485,366.003 119.033,365.894 119.581,365.784 
 120.129,365.675 120.677,365.566 121.225,365.457 121.772,365.347 122.32,365.238 122.868,365.129 123.416,365.02 123.964,364.911 124.512,364.801 125.06,364.692 
 125.607,364.583 126.155,364.474 126.703,364.364 127.251,364.255 127.799,364.146 128.347,364.037 128.894,363.928 129.442,363.818 129.99,363.709 130.538,363.6 
 131.086,363.491 131.634,363.382 132.181,363.272 132.729,363.163 133.277,363.054 133.825,362.945 134.373,362.835 134.921,362.726 135.469,362.617 136.016,362.508 
 136.564,362.399 137.112,362.289 137.66,362.18 138.208,362.071 138.756,361.962 139.303,361.852 139.851,361.743 140.399,361.634 140.947,361.525 141.495,361.416 
 142.043,361.306 142.59,361.124 143.138,360.942 143.686,360.76 144.234,360.578 144.782,360.396 145.33,360.214 145.877,360.032 146.425,359.85 146.973,359.668 
 147.521,359.486 148.069,359.304 148.617,359.122 149.165,358.94 149.712,358.758 150.26,358.576 150.808,358.394 151.356,358.212 151.904,358.03 152.452,357.848 
 152.999,357.666 153.547,357.484 154.095,357.302 154.643,357.12 155.191,356.938 155.739,356.756 156.286,356.574 156.834,356